The goal of this notebook is to scrape this website: 

https://www.tran.sla.ny.gov/JSP/query/PublicQueryPremisesSearchPage.jsp

For all of the liquor licenses in NYC, and then clean them.

## NYC Zip code list


from: http://nyc.pediacities.com/New_York_City_ZIP_Codes

In [2]:
manhattan = [10001,10002,10003,10004,10005,
             10006,10007,10009,10010,10011,
             10012,10013,10014,10016,10017,
             10018,10019,10020,10021,10022,
             10023,10024,10025,10026,10027,
             10028,10029,10030,10031,10032,
             10033,10034,10035,10036,10037,
             10038,10039,10040,10044,10048,
             10065,10069,10075,10111,10115,
             10128,10280,10281,10282]

In [3]:
brooklyn = [11201,11203,11204,11205,11206,
            11207,11208,11209,11210,11211,
            11212,11213,11214,11215,11216,
            11217,11218,11219,11220,11221,
            11222,11223,11224,11225,11226,
            11228,11229,11230,11231,11232,
            11233,11234,11235,11236,11237,
            11238,11239,11251]

In [4]:
queens = [11001,11004,11005,11040,11096,
          11101,11102,11103,11104,11105,
          11106,11109,11354,11355,11356,
          11357,11358,11359,11360,11361,
          11362,11363,11364,11365,11366,
          11367,11368,11369,11371,11372,
          11373,11374,11375,11377,11378,
          11379,11385,11411,11412,11413,
          11414,11415,11416,11417,11418,
          11419,11420,11421,11422,11423,
          11426,11427,11428,11429,11430,
          11432,11433,11434,11435,11436,
          11451,11691,11692,11694,11697]

In [5]:
bronx = [10451,10452,10453,10454,10455,
         10456,10457,10458,10459,10460,
         10461,10462,10465,10466,10467,
         10468,10469,10470,10471,10472,
         10473,10474,10475]

In [6]:
staten_island = [10301,10302,10303,10304,10305,10306,10307,10308,10309,10310,10312,10314]

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv
import requests
from bs4 import BeautifulSoup
import time

In [8]:
def get_liquor_licenses(zipcode, start=1, stop=None):
    
    driver = webdriver.Firefox()
    driver.get("https://www.tran.sla.ny.gov/JSP/query/PublicQueryPremisesSearchPage.jsp")
    elem = driver.find_element_by_name("zipCode")
    elem.send_keys(zipcode)
    elem.send_keys(Keys.RETURN)
    
    #Get the result table
    result_table = driver.find_element_by_class_name("default")
    row_vals = result_table.find_elements_by_tag_name("tr")
    
    #Start the clock!
    start_time = time.time()
    
    # Loop over each entry in each row
    col_vals = []
    print "There are {} rows\n".format(len(row_vals))
    for idx, row in enumerate(row_vals[start:stop]):
        if idx % 10 == 0:
            print idx + start, round(time.time() - start_time, 3)
        if idx > 0 and idx % 100 == 0:
            print idx + start
            csv_name = '{}_{}_{}.csv'.format(zipcode, start, idx + start)
            with open(csv_name, 'wb') as f:
                writer = csv.writer(f)
                writer.writerows(col_vals)
            
            
        columns = row.find_elements_by_class_name("displayvalue")
        
        #Crawl out to the link in the first entry - it contains filing date and other info
        license_name = columns[0].find_element_by_css_selector('a')
        license_link = license_name.get_attribute("href")
        r = requests.get(license_link)
        soup = BeautifulSoup(r.text, 'html.parser')
        values = soup.findAll("td", { "class" : "displayvalue" })
        # labels unnecessary
        #labels = soup.findAll("td", { "class" : "displaylabel" })
        # Clean values and labels
        #link_features = zip([val.contents for val in labels if "\xa0" not in str(val)][0:7],
        #                    [val.contents for val in values if "\xa0" not in str(val)][0:7])
        
        column_text = [col.text for col in columns]
        link_text = [val.string for val in values if "\xa0" not in str(val)][0:7]
        
        col_vals.append(column_text + link_text)
        
    driver.close()
    
    return col_vals


In [12]:
#Zip Loop!
for zip_code in manhattan[2:]:
    get_liquor_licenses(zip_code)

There are 2608 rows

1 0.0
11 10.798
21 21.012
31 33.028
41 43.434
51 54.59
61 64.793
71 78.005
81 89.004
91 99.765
101 111.907
101
111 128.864
121 144.574
131 156.07
141 167.662
151 179.281
161 191.386
171 203.145
181 213.931
191 224.374
201 235.419
201
211 245.615
221 255.948
231 266.913
241 277.826
251 288.225
261 299.47
271 309.544
281 322.331
291 333.212
301 344.352
301
311 358.091
321 370.059
331 380.02
341 390.969
351 401.444
361 411.822
371 423.118
381 434.423
391 445.866
401 456.291
401
411 467.243
421 478.798
431 491.21
441 502.004
451 512.594
461 523.818
471 534.263
481 544.979
491 560.079
501 570.944
501
511 584.129
521 594.768
531 605.515
541 616.88
551 749.899
561 760.17
571 771.721
581 781.831
591 792.746
601 803.135
601
611 816.521
621 832.171
631 842.341
641 852.201
651 862.755
661 873.716
671 884.43
681 897.17
691 908.73
701 918.798
701
711 930.163
721 941.554
731 953.874
741 965.695
751 977.518
761 988.756
771 1000.733
781 1012.582
791 1023.423
801 1035.409
801
811 1

WebDriverException: Message: "Can't load the profile. Profile Dir: /var/folders/02/hsxfg2bd7sx0sz0qwrtmfl9w0000gn/T/tmpD50n4E Firefox output: *** LOG addons.xpi: startup\n*** LOG addons.xpi: Skipping unavailable install location app-system-share\n*** LOG addons.xpi: checkForChanges\n*** LOG addons.xpi: No changes found\n" 